# GoDaddy Financial Services ML Models

This notebook contains three machine learning models focused on financial services aspects of GoDaddy's business:

1. **Customer Lifetime Value (CLV) Predictor** - Predicts future customer revenue potential
2. **Payment Failure Risk Model** - Identifies transactions at risk of failing
3. **Revenue Churn Prediction** - Predicts customers likely to reduce spending

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, sum as sf_sum, avg, count, datediff, current_date, lit, when
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.xgboost import XGBRegressor, XGBClassifier
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

session = get_active_session()
print(f"Connected to Snowflake: {session.get_current_database()}.{session.get_current_schema()}")

In [ ]:
session.sql("USE DATABASE GODADDY_INTELLIGENCE").collect()
session.sql("USE SCHEMA ANALYTICS").collect()
session.sql("USE WAREHOUSE GODADDY_WH").collect()
print("Context set to GODADDY_INTELLIGENCE.ANALYTICS")

---
## Model 1: Customer Lifetime Value (CLV) Predictor

Predicts the expected lifetime value of a customer based on their transaction history, product portfolio, and engagement patterns.

In [ ]:
clv_query = """
SELECT 
    c.CUSTOMER_ID,
    DATEDIFF('day', c.SIGNUP_DATE, CURRENT_DATE()) AS CUSTOMER_TENURE_DAYS,
    CASE c.CUSTOMER_SEGMENT 
        WHEN 'ENTERPRISE' THEN 2 
        WHEN 'SMALL_BUSINESS' THEN 1 
        ELSE 0 
    END AS SEGMENT_CODE,
    CASE WHEN c.IS_BUSINESS_CUSTOMER THEN 1 ELSE 0 END AS IS_BUSINESS,
    c.RISK_SCORE,
    COALESCE(c.TOTAL_DOMAINS, 0) AS TOTAL_DOMAINS,
    COALESCE(c.TOTAL_HOSTING_PLANS, 0) AS TOTAL_HOSTING_PLANS,
    COALESCE(c.TOTAL_SSL_CERTIFICATES, 0) AS TOTAL_SSL_CERTS,
    COALESCE(c.TOTAL_TRANSACTIONS, 0) AS TRANSACTION_COUNT,
    COALESCE(c.TOTAL_REVENUE, 0) AS HISTORICAL_REVENUE,
    COALESCE(c.TOTAL_SUPPORT_TICKETS, 0) AS SUPPORT_TICKETS,
    COALESCE(c.AVG_SATISFACTION_RATING, 3.5) AS AVG_SATISFACTION,
    c.LIFETIME_VALUE AS TARGET_LTV
FROM GODADDY_INTELLIGENCE.ANALYTICS.V_CUSTOMER_360 c
WHERE c.CUSTOMER_STATUS = 'ACTIVE'
    AND c.LIFETIME_VALUE IS NOT NULL
    AND c.LIFETIME_VALUE > 0
"""

clv_df = session.sql(clv_query)
print(f"CLV Training Data: {clv_df.count()} records")
clv_df.limit(5).to_pandas()

In [ ]:
feature_cols_clv = [
    'CUSTOMER_TENURE_DAYS', 'SEGMENT_CODE', 'IS_BUSINESS', 'RISK_SCORE',
    'TOTAL_DOMAINS', 'TOTAL_HOSTING_PLANS', 'TOTAL_SSL_CERTS',
    'TRANSACTION_COUNT', 'HISTORICAL_REVENUE', 'SUPPORT_TICKETS', 'AVG_SATISFACTION'
]
target_col_clv = 'TARGET_LTV'

clv_pandas = clv_df.to_pandas()
X_clv = clv_pandas[feature_cols_clv]
y_clv = clv_pandas[target_col_clv]

X_train_clv, X_test_clv, y_train_clv, y_test_clv = train_test_split(
    X_clv, y_clv, test_size=0.2, random_state=42
)
print(f"Training set: {len(X_train_clv)}, Test set: {len(X_test_clv)}")

In [ ]:
from xgboost import XGBRegressor as XGBRegressorLocal

clv_model = XGBRegressorLocal(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)
clv_model.fit(X_train_clv, y_train_clv)

y_pred_clv = clv_model.predict(X_test_clv)

mae_clv = mean_absolute_error(y_test_clv, y_pred_clv)
rmse_clv = np.sqrt(mean_squared_error(y_test_clv, y_pred_clv))
r2_clv = r2_score(y_test_clv, y_pred_clv)

print("=== CLV Model Performance ===")
print(f"MAE:  ${mae_clv:,.2f}")
print(f"RMSE: ${rmse_clv:,.2f}")
print(f"R²:   {r2_clv:.4f}")

In [ ]:
registry = Registry(session=session, database_name="GODADDY_INTELLIGENCE", schema_name="ANALYTICS")

sample_input_clv = X_train_clv.head(10)

clv_model_ref = registry.log_model(
    model=clv_model,
    model_name="CUSTOMER_LTV_PREDICTOR",
    version_name="V1",
    sample_input_data=sample_input_clv,
    comment="Predicts customer lifetime value based on tenure, product portfolio, and engagement metrics",
    metrics={
        "mae": mae_clv,
        "rmse": rmse_clv,
        "r2_score": r2_clv
    }
)
print(f"CLV Model registered: {clv_model_ref.fully_qualified_model_name}")

---
## Model 2: Payment Failure Risk Model

Classifies transactions by their likelihood of payment failure based on customer history, payment method, and transaction patterns.

In [ ]:
payment_query = """
WITH customer_payment_history AS (
    SELECT 
        CUSTOMER_ID,
        COUNT(*) AS total_transactions,
        SUM(CASE WHEN PAYMENT_STATUS = 'FAILED' THEN 1 ELSE 0 END) AS failed_count,
        SUM(CASE WHEN PAYMENT_STATUS = 'REFUNDED' THEN 1 ELSE 0 END) AS refund_count,
        AVG(TOTAL_AMOUNT) AS avg_transaction_amount
    FROM GODADDY_INTELLIGENCE.RAW.TRANSACTIONS
    GROUP BY CUSTOMER_ID
)
SELECT 
    t.TRANSACTION_ID,
    DATEDIFF('day', c.SIGNUP_DATE, t.TRANSACTION_DATE) AS DAYS_AS_CUSTOMER,
    CASE c.CUSTOMER_SEGMENT 
        WHEN 'ENTERPRISE' THEN 2 
        WHEN 'SMALL_BUSINESS' THEN 1 
        ELSE 0 
    END AS SEGMENT_CODE,
    c.RISK_SCORE,
    CASE t.PAYMENT_METHOD
        WHEN 'CREDIT_CARD' THEN 0
        WHEN 'PAYPAL' THEN 1
        WHEN 'BANK_TRANSFER' THEN 2
        ELSE 3
    END AS PAYMENT_METHOD_CODE,
    CASE t.TRANSACTION_TYPE
        WHEN 'DOMAIN_REGISTRATION' THEN 0
        WHEN 'DOMAIN_RENEWAL' THEN 1
        WHEN 'HOSTING_PURCHASE' THEN 2
        WHEN 'HOSTING_RENEWAL' THEN 3
        ELSE 4
    END AS TRANSACTION_TYPE_CODE,
    t.TOTAL_AMOUNT,
    t.DISCOUNT_AMOUNT,
    COALESCE(h.total_transactions, 0) AS PRIOR_TRANSACTIONS,
    COALESCE(h.failed_count, 0) AS PRIOR_FAILURES,
    COALESCE(h.refund_count, 0) AS PRIOR_REFUNDS,
    COALESCE(h.avg_transaction_amount, 0) AS AVG_PRIOR_AMOUNT,
    CASE WHEN t.PAYMENT_STATUS = 'FAILED' THEN 1 ELSE 0 END AS IS_FAILED
FROM GODADDY_INTELLIGENCE.RAW.TRANSACTIONS t
JOIN GODADDY_INTELLIGENCE.RAW.CUSTOMERS c ON t.CUSTOMER_ID = c.CUSTOMER_ID
LEFT JOIN customer_payment_history h ON t.CUSTOMER_ID = h.CUSTOMER_ID
WHERE t.PAYMENT_STATUS IN ('COMPLETED', 'FAILED')
"""

payment_df = session.sql(payment_query)
print(f"Payment Risk Training Data: {payment_df.count()} records")
payment_df.limit(5).to_pandas()

In [ ]:
feature_cols_payment = [
    'DAYS_AS_CUSTOMER', 'SEGMENT_CODE', 'RISK_SCORE', 'PAYMENT_METHOD_CODE',
    'TRANSACTION_TYPE_CODE', 'TOTAL_AMOUNT', 'DISCOUNT_AMOUNT',
    'PRIOR_TRANSACTIONS', 'PRIOR_FAILURES', 'PRIOR_REFUNDS', 'AVG_PRIOR_AMOUNT'
]
target_col_payment = 'IS_FAILED'

payment_pandas = payment_df.to_pandas()
X_payment = payment_pandas[feature_cols_payment]
y_payment = payment_pandas[target_col_payment]

print(f"Class distribution: Failed={y_payment.sum()}, Success={len(y_payment) - y_payment.sum()}")

X_train_pay, X_test_pay, y_train_pay, y_test_pay = train_test_split(
    X_payment, y_payment, test_size=0.2, random_state=42, stratify=y_payment
)
print(f"Training set: {len(X_train_pay)}, Test set: {len(X_test_pay)}")

In [ ]:
from xgboost import XGBClassifier as XGBClassifierLocal

scale_pos_weight = (len(y_train_pay) - y_train_pay.sum()) / max(y_train_pay.sum(), 1)

payment_model = XGBClassifierLocal(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)
payment_model.fit(X_train_pay, y_train_pay)

y_pred_pay = payment_model.predict(X_test_pay)
y_pred_proba_pay = payment_model.predict_proba(X_test_pay)[:, 1]

acc_pay = accuracy_score(y_test_pay, y_pred_pay)
prec_pay = precision_score(y_test_pay, y_pred_pay, zero_division=0)
rec_pay = recall_score(y_test_pay, y_pred_pay, zero_division=0)
f1_pay = f1_score(y_test_pay, y_pred_pay, zero_division=0)
auc_pay = roc_auc_score(y_test_pay, y_pred_proba_pay) if y_test_pay.sum() > 0 else 0

print("=== Payment Failure Risk Model Performance ===")
print(f"Accuracy:  {acc_pay:.4f}")
print(f"Precision: {prec_pay:.4f}")
print(f"Recall:    {rec_pay:.4f}")
print(f"F1 Score:  {f1_pay:.4f}")
print(f"AUC-ROC:   {auc_pay:.4f}")

In [ ]:
sample_input_payment = X_train_pay.head(10)

payment_model_ref = registry.log_model(
    model=payment_model,
    model_name="PAYMENT_FAILURE_RISK",
    version_name="V1",
    sample_input_data=sample_input_payment,
    comment="Classifies transactions by payment failure risk based on customer history and transaction patterns",
    metrics={
        "accuracy": acc_pay,
        "precision": prec_pay,
        "recall": rec_pay,
        "f1_score": f1_pay,
        "auc_roc": auc_pay
    }
)
print(f"Payment Risk Model registered: {payment_model_ref.fully_qualified_model_name}")

---
## Model 3: Revenue Churn Prediction Model

Identifies customers at risk of reducing their spending (revenue churn) based on engagement trends and service utilization.

In [ ]:
churn_query = """
WITH recent_activity AS (
    SELECT 
        CUSTOMER_ID,
        SUM(CASE WHEN TRANSACTION_DATE >= DATEADD('month', -3, CURRENT_DATE()) THEN TOTAL_AMOUNT ELSE 0 END) AS REVENUE_LAST_3M,
        SUM(CASE WHEN TRANSACTION_DATE >= DATEADD('month', -6, CURRENT_DATE()) 
                 AND TRANSACTION_DATE < DATEADD('month', -3, CURRENT_DATE()) THEN TOTAL_AMOUNT ELSE 0 END) AS REVENUE_PRIOR_3M,
        COUNT(CASE WHEN TRANSACTION_DATE >= DATEADD('month', -3, CURRENT_DATE()) THEN 1 END) AS TXN_COUNT_LAST_3M,
        MAX(TRANSACTION_DATE) AS LAST_TRANSACTION_DATE
    FROM GODADDY_INTELLIGENCE.RAW.TRANSACTIONS
    WHERE PAYMENT_STATUS = 'COMPLETED'
    GROUP BY CUSTOMER_ID
),
support_activity AS (
    SELECT 
        CUSTOMER_ID,
        COUNT(*) AS TICKETS_LAST_6M,
        AVG(SATISFACTION_RATING) AS AVG_RECENT_SATISFACTION
    FROM GODADDY_INTELLIGENCE.RAW.SUPPORT_TICKETS
    WHERE CREATED_DATE >= DATEADD('month', -6, CURRENT_DATE())
    GROUP BY CUSTOMER_ID
),
domain_status AS (
    SELECT 
        CUSTOMER_ID,
        COUNT(*) AS TOTAL_DOMAINS,
        SUM(CASE WHEN RENEWAL_STATUS IN ('EXPIRED', 'PENDING_RENEWAL') THEN 1 ELSE 0 END) AS DOMAINS_AT_RISK,
        SUM(CASE WHEN AUTO_RENEW_ENABLED = FALSE THEN 1 ELSE 0 END) AS DOMAINS_NO_AUTORENEW
    FROM GODADDY_INTELLIGENCE.RAW.DOMAINS
    GROUP BY CUSTOMER_ID
)
SELECT 
    c.CUSTOMER_ID,
    DATEDIFF('day', c.SIGNUP_DATE, CURRENT_DATE()) AS TENURE_DAYS,
    CASE c.CUSTOMER_SEGMENT 
        WHEN 'ENTERPRISE' THEN 2 
        WHEN 'SMALL_BUSINESS' THEN 1 
        ELSE 0 
    END AS SEGMENT_CODE,
    c.RISK_SCORE,
    COALESCE(r.REVENUE_LAST_3M, 0) AS REVENUE_LAST_3M,
    COALESCE(r.REVENUE_PRIOR_3M, 0) AS REVENUE_PRIOR_3M,
    COALESCE(r.TXN_COUNT_LAST_3M, 0) AS TXN_COUNT_LAST_3M,
    COALESCE(DATEDIFF('day', r.LAST_TRANSACTION_DATE, CURRENT_DATE()), 365) AS DAYS_SINCE_LAST_TXN,
    COALESCE(s.TICKETS_LAST_6M, 0) AS SUPPORT_TICKETS_6M,
    COALESCE(s.AVG_RECENT_SATISFACTION, 3.5) AS AVG_RECENT_SATISFACTION,
    COALESCE(d.TOTAL_DOMAINS, 0) AS TOTAL_DOMAINS,
    COALESCE(d.DOMAINS_AT_RISK, 0) AS DOMAINS_AT_RISK,
    COALESCE(d.DOMAINS_NO_AUTORENEW, 0) AS DOMAINS_NO_AUTORENEW,
    CASE 
        WHEN COALESCE(r.REVENUE_PRIOR_3M, 0) > 0 
             AND COALESCE(r.REVENUE_LAST_3M, 0) < COALESCE(r.REVENUE_PRIOR_3M, 0) * 0.5 THEN 1
        WHEN COALESCE(r.REVENUE_LAST_3M, 0) = 0 AND COALESCE(r.REVENUE_PRIOR_3M, 0) > 0 THEN 1
        ELSE 0
    END AS IS_CHURNING
FROM GODADDY_INTELLIGENCE.RAW.CUSTOMERS c
LEFT JOIN recent_activity r ON c.CUSTOMER_ID = r.CUSTOMER_ID
LEFT JOIN support_activity s ON c.CUSTOMER_ID = s.CUSTOMER_ID
LEFT JOIN domain_status d ON c.CUSTOMER_ID = d.CUSTOMER_ID
WHERE c.CUSTOMER_STATUS = 'ACTIVE'
    AND c.SIGNUP_DATE < DATEADD('month', -6, CURRENT_DATE())
"""

churn_df = session.sql(churn_query)
print(f"Revenue Churn Training Data: {churn_df.count()} records")
churn_df.limit(5).to_pandas()

In [ ]:
feature_cols_churn = [
    'TENURE_DAYS', 'SEGMENT_CODE', 'RISK_SCORE', 'REVENUE_LAST_3M', 'REVENUE_PRIOR_3M',
    'TXN_COUNT_LAST_3M', 'DAYS_SINCE_LAST_TXN', 'SUPPORT_TICKETS_6M',
    'AVG_RECENT_SATISFACTION', 'TOTAL_DOMAINS', 'DOMAINS_AT_RISK', 'DOMAINS_NO_AUTORENEW'
]
target_col_churn = 'IS_CHURNING'

churn_pandas = churn_df.to_pandas()
X_churn = churn_pandas[feature_cols_churn]
y_churn = churn_pandas[target_col_churn]

print(f"Class distribution: Churning={y_churn.sum()}, Active={len(y_churn) - y_churn.sum()}")

X_train_churn, X_test_churn, y_train_churn, y_test_churn = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=42, stratify=y_churn
)
print(f"Training set: {len(X_train_churn)}, Test set: {len(X_test_churn)}")

In [ ]:
scale_pos_weight_churn = (len(y_train_churn) - y_train_churn.sum()) / max(y_train_churn.sum(), 1)

churn_model = XGBClassifierLocal(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight_churn,
    random_state=42
)
churn_model.fit(X_train_churn, y_train_churn)

y_pred_churn = churn_model.predict(X_test_churn)
y_pred_proba_churn = churn_model.predict_proba(X_test_churn)[:, 1]

acc_churn = accuracy_score(y_test_churn, y_pred_churn)
prec_churn = precision_score(y_test_churn, y_pred_churn, zero_division=0)
rec_churn = recall_score(y_test_churn, y_pred_churn, zero_division=0)
f1_churn = f1_score(y_test_churn, y_pred_churn, zero_division=0)
auc_churn = roc_auc_score(y_test_churn, y_pred_proba_churn) if y_test_churn.sum() > 0 else 0

print("=== Revenue Churn Model Performance ===")
print(f"Accuracy:  {acc_churn:.4f}")
print(f"Precision: {prec_churn:.4f}")
print(f"Recall:    {rec_churn:.4f}")
print(f"F1 Score:  {f1_churn:.4f}")
print(f"AUC-ROC:   {auc_churn:.4f}")

In [ ]:
sample_input_churn = X_train_churn.head(10)

churn_model_ref = registry.log_model(
    model=churn_model,
    model_name="REVENUE_CHURN_PREDICTOR",
    version_name="V1",
    sample_input_data=sample_input_churn,
    comment="Identifies customers at risk of revenue churn based on spending trends and engagement patterns",
    metrics={
        "accuracy": acc_churn,
        "precision": prec_churn,
        "recall": rec_churn,
        "f1_score": f1_churn,
        "auc_roc": auc_churn
    }
)
print(f"Revenue Churn Model registered: {churn_model_ref.fully_qualified_model_name}")

---
## Model Summary

All three financial services ML models have been trained and registered in the Snowflake Model Registry.

In [ ]:
print("="*60)
print("REGISTERED MODELS SUMMARY")
print("="*60)

models = registry.show_models()
for model in models.to_pandas().itertuples():
    print(f"\nModel: {model.name}")
    print(f"  Database: {model.database_name}")
    print(f"  Schema: {model.schema_name}")

print("\n" + "="*60)
print("MODELS READY FOR AGENT INTEGRATION")
print("="*60)
print("\n1. CUSTOMER_LTV_PREDICTOR - Predict customer lifetime value")
print("2. PAYMENT_FAILURE_RISK - Assess payment failure probability")
print("3. REVENUE_CHURN_PREDICTOR - Identify revenue churn risk")